# 0. Prepare

In [1]:
import sys
sys.path.append('../../dataset/')
sys.path.append('../../network/')
sys.path.append('../../model/')

import os
import glob
import time
import torch
import joblib
import logging
import argparse
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch
import seaborn as sns

from pathlib import Path
from main_loading import *
from main_network import *
from main_model_rec import *
from main_model_one_class import *
from scipy.spatial import KDTree
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [2]:
identifier = 1

In [3]:
device = 'cuda:1'
root = '/net/leksai/data/FashionMNIST'
rec_model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(identifier)
oc_model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[{}]_[]_[1]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'.format(identifier)

# 1. Load Only the Encoder Part

> ## For One-Class Model

In [4]:
class OneClassEncoder:
    def __init__(self):
        self.net = None
        self.net_name = None

    def set_network(self, net_name):
        self.net_name = net_name
        self.net = build_network(net_name)

    def load_model(self, model_path, map_location):
        model_dict = torch.load(model_path, map_location=map_location)
        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])

    def test(self, train, dataset, device, batch_size, n_jobs_dataloader):
        if train:
            all_loader, _ = dataset.loaders(batch_size=batch_size,
                                            num_workers=n_jobs_dataloader)
        else:
            all_loader = dataset.loaders(batch_size=batch_size,
                                         num_workers=n_jobs_dataloader)
        net = self.net.to(device)
        criterion = nn.MSELoss(reduction='none')
        
        n_batches = 0
        X_pred_list = []
        net.eval()
        
        with torch.no_grad():
            for data in all_loader:
                X, y, idx = data
                X, y, idx = X.to(device), y.to(device), idx.to(device)

                X_pred = net(X)
                X_pred_list += X_pred
        
        return np.array(X_pred_list)

In [5]:
oc_encoder = OneClassEncoder()
oc_encoder.set_network('fmnist_LeNet_one_class')
oc_encoder.load_model(oc_model_path, device)

> ## For Reconstruction Model

In [6]:
class RecEncoder:
    def __init__(self):

        self.net_name = None
        self.net = None
        self.ae_net = None


    def set_network(self, net_name: str='fmnist_LeNet_one_class'):
        """
        Set the network structure for the model.
        The key here is to initialize <self.net>.
        """
        self.net_name = net_name
        self.net = build_network(net_name)
        self.ae_net = build_network('fmnist_LeNet_rec')

    def load_model(self,
                   model_path,
                   map_location='cuda:1'):
        """
        Load the trained model for the model.
        The key here is to initialize <self.c>.
        """
        # Load the general model
        model_dict = torch.load(model_path, map_location=map_location)
        self.ae_net.load_state_dict(model_dict['net_dict'])
        
        # Obtain the net dictionary
        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()
        
        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)

        # Load the new state_dict
        self.net.load_state_dict(net_dict)
        

    def save_model(self, export_model, save_ae=True):
        net_dict = self.net.state_dict()
        torch.save({'net_dict': net_dict}, export_model)
    
    def test(self, train, dataset, device, batch_size, n_jobs_dataloader):
        if train:
            all_loader, _ = dataset.loaders(batch_size=batch_size,
                                            num_workers=n_jobs_dataloader)
        else:
            all_loader = dataset.loaders(batch_size=batch_size,
                                         num_workers=n_jobs_dataloader)
        net = self.net.to(device)
        criterion = nn.MSELoss(reduction='none')
        
        n_batches = 0
        X_pred_list = []
        net.eval()
        
        with torch.no_grad():
            for data in all_loader:
                X, y, idx = data
                X, y, idx = X.to(device), y.to(device), idx.to(device)

                X_pred = net(X)
                X_pred_list += X_pred
        
        return np.array(X_pred_list)

In [7]:
rec_encoder = RecEncoder()
rec_encoder.set_network()
rec_encoder.load_model(rec_model_path, device)

# 2. Dataset Loading

In [8]:
dataset_dict_train = {}
dataset_dict_test = {}
name_list = ['tshirt', 'trouser', 'pullover', 'dress', 'coat',
             'sandal', 'shirt', 'sneaker', 'bag', 'boot']

In [9]:
for i, name in enumerate(name_list):
    dataset_dict_train[name] = load_dataset(loader_name='fmnist',
                                            root=root,
                                            label_normal=(i,),
                                            ratio_abnormal=0)

Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!


In [10]:
for i, name in enumerate(name_list):
    dataset_dict_test[name] = load_dataset(loader_name='fmnist_eval',
                                            root=root,
                                            label_eval=(i,),
                                            test_eval=True)

# 3. Latent Vector Obtaining

> Let's use the unsupervised model of `pullover` as feature extractor, meaning that the latent space are defined from the neural weights of the unsupervised model of `pullover`.

In [11]:
latent_dict_train = {'oc':{}, 'rec':{}}
latent_dict_test = {'oc':{}, 'rec':{}}

> ## For One Class Model

In [12]:
for name in name_list:
    dataset_train = dataset_dict_train[name]
    data_train = oc_encoder.test(True, dataset_train, device, 6000, 0)
    data_train = np.array([x.cpu().numpy() for x in data_train])
    latent_dict_train['oc'][name] = data_train

In [13]:
for name in name_list:
    dataset_test = dataset_dict_test[name]
    data_test = oc_encoder.test(False, dataset_test, device, 1000, 0)
    data_test = np.array([x.cpu().numpy() for x in data_test])
    latent_dict_test['oc'][name] = data_test

> ## For Rec Model

In [14]:
for name in name_list:
    dataset_train = dataset_dict_train[name]
    data_train = rec_encoder.test(True, dataset_train, device, 6000, 0)
    data_train = np.array([x.cpu().numpy() for x in data_train])
    latent_dict_train['rec'][name] = data_train

In [15]:
for name in name_list:
    dataset_test = dataset_dict_test[name]
    data_test = rec_encoder.test(False, dataset_test, device, 1000, 0)
    data_test = np.array([x.cpu().numpy() for x in data_test])
    latent_dict_test['rec'][name] = data_test

# Function for KL Divergence by KNN

> Cited from https://github.com/nhartland/KL-divergence-estimators

In [16]:
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree

def knn_distance(point, sample, k):
    """ 
    Euclidean distance from `point` to it's `k`-Nearest
    Neighbour in `sample` 
    """
    norms = np.linalg.norm(sample-point, axis=1)
    return np.sort(norms)[k]


def verify_sample_shapes(s1, s2, k):
    # Expects [N, D]
    assert(len(s1.shape) == len(s2.shape) == 2)
    # Check dimensionality of sample is identical
    assert(s1.shape[1] == s2.shape[1])
    
    
def skl_estimator(s1, s2, k=3):
    """ 
    KL-Divergence estimator using scikit-learn's NearestNeighbours.
    Inputs:
        s1: (N_1,D) Sample drawn from distribution P
        s2: (N_2,D) Sample drawn from distribution Q
        k: Number of neighbours considered (default 1)
    return: 
        estimated D(P|Q)
    """
    verify_sample_shapes(s1, s2, k)

    n, m = len(s1), len(s2)
    d = float(s1.shape[1])
    D = np.log(m / (n - 1))

    s1_neighbourhood = NearestNeighbors(k + 1, 10).fit(s1)
    s2_neighbourhood = NearestNeighbors(k, 10).fit(s2)

    for p1 in s1:
        s1_distances, indices = s1_neighbourhood.kneighbors([p1], k + 1)
        s2_distances, indices = s2_neighbourhood.kneighbors([p1], k)
        rho = s1_distances[0][- 1]
        nu = s2_distances[0][- 1]
        D += (d / n) * np.log(nu / rho)
        D += 0
    return D

# Calculating Joint KL Divergence for Reconstruction Model

In [17]:
name_list_ = name_list[:]
identifier_name = name_list_.pop(identifier)

In [18]:
div_joint = {k: {} for k in name_list_}
div_margin = {k: {} for k in name_list_}

In [19]:
div_margin_train = {k: {} for k in name_list_}

In [20]:
# Fix normal train, normal test.
normal_train = latent_dict_train['rec'][identifier_name]
normal_test = latent_dict_test['rec'][identifier_name]

In [21]:
# Fix abnormal test
# abnormal_test = latent_dict_test['rec']['shirt']
# div_joint_test = div_joint['shirt']
# div_margin_test = div_margin['shirt']

## Calculate for $KL(P(normal train) & P(abnormal train i) || P(normal test) & P(abnormal test))$

In [22]:
name_list_

['tshirt',
 'pullover',
 'dress',
 'coat',
 'sandal',
 'shirt',
 'sneaker',
 'bag',
 'boot']

In [66]:
for test_j in name_list_:
    print('Outer loop', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    div_joint_test = div_joint[test_j]
    div_margin_test = div_margin[test_j]
    
    for i in name_list_:
        print(i)
        abnormal_train_i = latent_dict_train['rec'][i]

        # Calculating for joint divergence
        left_joint = np.r_[normal_train, abnormal_train_i]
        right_joint = np.r_[normal_test, abnormal_test]
        div_joint_test[i] = .5 * (skl_estimator(left_joint, right_joint) + 
                                  skl_estimator(right_joint, left_joint))
        print('KL joint:', div_joint_test[i])

        # Calculating for marginal divergence
        left_margin = normal_train
        right_margin = abnormal_train_i
        div_margin_test[i] = .5 * (skl_estimator(left_margin, right_margin) + 
                                   skl_estimator(right_margin, left_margin))
        print('KL marginal:', div_margin_test[i]) 

Outer loop tshirt
tshirt
KL joint: 0.04297313733503394
KL marginal: 42.44060782296498
pullover
KL joint: 12.940294187072718
KL marginal: 46.04073123243284
dress
KL joint: 9.059777155202427
KL marginal: 23.661147154746285
coat
KL joint: 14.976308234296113
KL marginal: 40.4762669160649
sandal
KL joint: 17.879083780683303
KL marginal: 64.49535698096939
shirt
KL joint: 5.955988815030759
KL marginal: 40.12929516160251
sneaker
KL joint: 23.353730733429067
KL marginal: 83.39788756270843
bag
KL joint: 16.524613016708642
KL marginal: 51.50970617822665
boot
KL joint: 24.333737229977746
KL marginal: 75.89092252077711
Outer loop pullover
tshirt
KL joint: 13.037469163581049
KL marginal: 42.44060782296498
pullover
KL joint: 0.11310026403991369
KL marginal: 46.04073123243284
dress
KL joint: 10.009726827947503
KL marginal: 23.661147154746285
coat
KL joint: 3.811402486251397
KL marginal: 40.4762669160649
sandal
KL joint: 17.17025043992158
KL marginal: 64.49535698096939
shirt
KL joint: 4.178893536455789

In [67]:
joblib.dump(div_joint, 'div_joint.pkl')
joblib.dump(div_margin, 'div_margin.pkl')

['div_margin.pkl']

In [68]:
for test_j in name_list_:
    print('Outer loop:', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    div_margin_train_test = div_margin_train[test_j]
    
    for i in name_list_:
        print(i)
        abnormal_train_i = latent_dict_train['rec'][i]

        # Calculating for marginal divergence
        left = abnormal_train_i
        right = abnormal_test
        div_margin_train_test[i] = .5 * (skl_estimator(left, right) + 
                                         skl_estimator(right, left))
        print('KL marginal:', div_margin_train_test[i]) 

Outer loop: tshirt
tshirt
KL marginal: -0.15133535256649733
pullover
KL marginal: 26.73443896527209
dress
KL marginal: 22.548956410777024
coat
KL marginal: 31.4119649317732
sandal
KL marginal: 46.88942393483737
shirt
KL marginal: 12.013741028158018
sneaker
KL marginal: 63.24370683882583
bag
KL marginal: 36.42932009991142
boot
KL marginal: 61.70334518694017
Outer loop: pullover
tshirt
KL marginal: 27.70204126867308
pullover
KL marginal: 0.0006826764006686581
dress
KL marginal: 30.194767231932218
coat
KL marginal: 7.739614989914434
sandal
KL marginal: 45.084534082798896
shirt
KL marginal: 8.665229561059736
sneaker
KL marginal: 62.46027498269018
bag
KL marginal: 27.071715322320777
boot
KL marginal: 59.041532150326205
Outer loop: dress
tshirt
KL marginal: 21.85332785274322
pullover
KL marginal: 28.800527300037306
dress
KL marginal: 0.17221017527233506
coat
KL marginal: 24.19889873176463
sandal
KL marginal: 46.14076125203323
shirt
KL marginal: 20.90197478712808
sneaker
KL marginal: 64.88131

In [69]:
joblib.dump(div_margin_train, 'div_margin_train.pkl')

['div_margin_train.pkl']

In [23]:
k = joblib.load('div_margin_train.pkl')

In [24]:
k

{'tshirt': {'tshirt': -0.15133535256649733,
  'pullover': 26.73443896527209,
  'dress': 22.548956410777024,
  'coat': 31.4119649317732,
  'sandal': 46.88942393483737,
  'shirt': 12.013741028158018,
  'sneaker': 63.24370683882583,
  'bag': 36.42932009991142,
  'boot': 61.70334518694017},
 'pullover': {'tshirt': 27.70204126867308,
  'pullover': 0.0006826764006686581,
  'dress': 30.194767231932218,
  'coat': 7.739614989914434,
  'sandal': 45.084534082798896,
  'shirt': 8.665229561059736,
  'sneaker': 62.46027498269018,
  'bag': 27.071715322320777,
  'boot': 59.041532150326205},
 'dress': {'tshirt': 21.85332785274322,
  'pullover': 28.800527300037306,
  'dress': 0.17221017527233506,
  'coat': 24.19889873176463,
  'sandal': 46.14076125203323,
  'shirt': 20.90197478712808,
  'sneaker': 64.88131191232239,
  'bag': 33.88503987521678,
  'boot': 58.12364431389621},
 'coat': {'tshirt': 31.24545504912384,
  'pullover': 7.2269451403715275,
  'dress': 25.498560306724595,
  'coat': -0.070610893349754

## Calculate for $KL(P(normal test) || P(abnormal test_{i}))$

In [70]:
margin_test_for_identifier = []

In [71]:
normal_test = latent_dict_test['rec'][identifier_name]

In [72]:
for test_j in name_list:
    print(test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    left = normal_test
    right = abnormal_test
    div = .5 * (skl_estimator(left, right) + 
                skl_estimator(right, left))
    margin_test_for_identifier.append(div)
    print(div)

tshirt
39.157163500205755
trouser
-3.185809675660219
pullover
45.82225416247262
dress
18.926312129634113
coat
38.646466931878514
sandal
60.00383754227329
shirt
36.58036591143738
sneaker
78.00321405045031
bag
50.22903510799573
boot
72.73385976938161


In [73]:
joblib.dump(margin_test_for_identifier, 'margin_test_for_identifier.pkl')

['margin_test_for_identifier.pkl']

# Calculating Joint KL Divergence for One Class Model

In [74]:
div_joint_oc = {k: {} for k in name_list_}
div_margin_oc = {k: {} for k in name_list_}

# Fix normal train, normal test.
normal_train = latent_dict_train['oc'][identifier_name]
normal_test = latent_dict_test['oc'][identifier_name]

In [75]:
for test_j in name_list_:
    print('Outer loop', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    div_joint_test_oc = div_joint_oc[test_j]
    div_margin_test_oc = div_margin_oc[test_j]
    
    for i in name_list_:
        print(i)
        abnormal_train_i = latent_dict_train['oc'][i]

        # Calculating for joint divergence
        left_joint = np.r_[normal_train, abnormal_train_i]
        right_joint = np.r_[normal_test, abnormal_test]
        div_joint_test_oc[i] = .5 * (skl_estimator(left_joint, right_joint) + 
                                     skl_estimator(right_joint, left_joint))
        print('KL joint:', div_joint_test_oc[i])

        # Calculating for marginal divergence
        left_margin = normal_train
        right_margin = abnormal_train_i
        div_margin_test_oc[i] = .5 * (skl_estimator(left_margin, right_margin) + 
                                      skl_estimator(right_margin, left_margin))
        print('KL marginal:', div_margin_test_oc[i]) 

Outer loop tshirt
tshirt
KL joint: 0.06442656597177754
KL marginal: 29.085623417038434
pullover
KL joint: 10.851604057391224
KL marginal: 34.044819819125074
dress
KL joint: 6.965334260836926
KL marginal: 15.216839277944953
coat
KL joint: 10.818900058379816
KL marginal: 30.373216371566084
sandal
KL joint: 14.351057740542831
KL marginal: 51.73131364967841
shirt
KL joint: 4.626827691428772
KL marginal: 27.095528122444087
sneaker
KL joint: 21.45130728679208
KL marginal: 64.84605050234873
bag
KL joint: 14.157942364089664
KL marginal: 44.412671268232636
boot
KL joint: 16.476586251851
KL marginal: 58.11750817684728
Outer loop pullover
tshirt
KL joint: 10.911093189310066
KL marginal: 29.085623417038434
pullover
KL joint: 0.13598933846877648
KL marginal: 34.044819819125074
dress
KL joint: 7.516029821336033
KL marginal: 15.216839277944953
coat
KL joint: 2.99948168486371
KL marginal: 30.373216371566084
sandal
KL joint: 13.468213110900965
KL marginal: 51.73131364967841
shirt
KL joint: 2.9192544406

In [76]:
joblib.dump(div_joint_oc, 'div_joint_oc.pkl')
joblib.dump(div_margin_oc, 'div_margin_oc.pkl')

['div_margin_oc.pkl']

In [77]:
margin_test_for_identifier_oc = []
normal_test = latent_dict_test['oc'][identifier_name]

for test_j in name_list:
    print(test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    left = normal_test
    right = abnormal_test
    div = .5 * (skl_estimator(left, right) + 
                skl_estimator(right, left))
    margin_test_for_identifier_oc.append(div)
    print(div)

tshirt
22.728053641835814
trouser
-1.6615655741299602
pullover
31.811143414276163
dress
12.640762049414796
coat
27.539633871800493
sandal
44.93805667167079
shirt
23.165203686743897
sneaker
61.30729141449248
bag
41.85737305281228
boot
51.941154148560244


In [78]:
joblib.dump(margin_test_for_identifier_oc, 'margin_test_for_identifier_oc.pkl')

['margin_test_for_identifier_oc.pkl']

In [79]:
div_margin_train_oc = {k: {} for k in name_list_}

In [80]:
for test_j in name_list_:
    print('Outer loop:', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    div_margin_train_test_oc = div_margin_train_oc[test_j]
    
    for i in name_list_:
        print(i)
        abnormal_train_i = latent_dict_train['oc'][i]

        # Calculating for marginal divergence
        left = abnormal_train_i
        right = abnormal_test
        div_margin_train_test_oc[i] = .5 * (skl_estimator(left, right) + 
                                            skl_estimator(right, left))
        print('KL marginal:', div_margin_train_test_oc[i]) 

Outer loop: tshirt
tshirt
KL marginal: 0.10247373627478584
pullover
KL marginal: 23.72451306665972
dress
KL marginal: 16.753608583526226
coat
KL marginal: 24.23756937428399
sandal
KL marginal: 37.46523967322126
shirt
KL marginal: 10.189451080252715
sneaker
KL marginal: 53.85162837889534
bag
KL marginal: 33.019658404951585
boot
KL marginal: 42.077061395467986
Outer loop: pullover
tshirt
KL marginal: 24.49589036576935
pullover
KL marginal: 0.17657203408126243
dress
KL marginal: 22.537507609106676
coat
KL marginal: 6.8369520418339995
sandal
KL marginal: 36.251387077296116
shirt
KL marginal: 6.674165021524672
sneaker
KL marginal: 52.44686758719365
bag
KL marginal: 29.02224500511624
boot
KL marginal: 40.31664460893286
Outer loop: dress
tshirt
KL marginal: 17.703257818502024
pullover
KL marginal: 21.99369172512361
dress
KL marginal: 0.12339850499733895
coat
KL marginal: 16.411710104282587
sandal
KL marginal: 34.07335173162171
shirt
KL marginal: 15.762244360616407
sneaker
KL marginal: 48.9765

In [81]:
joblib.dump(div_margin_train_oc, 'div_margin_train_oc.pkl')

['div_margin_train_oc.pkl']